# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Part 1: Load and Create Dataframe from Wiki Page¶

#### Create a dataframe from the csv file

In [2]:
# The code was removed by Watson Studio for sharing.

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


#### Remove cities that is not capital of a country/Not a primary type capital

In [3]:
long_capital.loc[long_capital['capital'] != 'primary','capital'] = np.nan
long_capital.dropna(inplace=True)
long_capital.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
9,Dhaka,Dhaka,23.7231,90.4086,Bangladesh,BD,BGD,Dhaka,primary,12797394.0,1050529279
10,Buenos Aires,Buenos Aires,-34.6025,-58.3975,Argentina,AR,ARG,"Buenos Aires, Ciudad Autónoma de",primary,12795000.0,1032717330
12,Cairo,Cairo,30.0500,31.2500,Egypt,EG,EGY,Al Qāhirah,primary,11893000.0,1818253931


#### New dataframe without unnecessary columns 

In [37]:
capital=long_capital[['country','city','lat','lng']]
capital=capital.rename(columns={"city": "capital"})
capital=capital.reset_index()
capital=capital.drop(columns=['index'])
capital.head()

,country,capital,lat,lng
0,Japan,Tokyo,35.6850,139.7514
1,Mexico,Mexico City,19.4424,-99.1310
2,Bangladesh,Dhaka,23.7231,90.4086
3,Argentina,Buenos Aires,-34.6025,-58.3975
4,Egypt,Cairo,30.0500,31.2500


In [40]:
capital.shape

(193, 4)

In [5]:
# create map of world using latitude and longitude values
map_world = folium.Map(location=[0, 0], zoom_start=3)

# add markers to map
for lat, lng, cap,country in zip(capital['lat'], capital['lng'], capital['capital'],capital['country']):
    label = '{}, {}'.format(cap, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        clustered_marker = True,
        parse_html=False).add_to(map_world)
  
    
map_world

### Foursquare Credentials

In [6]:
CLIENT_ID = 'RMXFBEIKQ4VQJO50BKBC5XXEMCATBDENIXCDJG1GTFJWSSIH' # your Foursquare ID
CLIENT_SECRET = 'AVJVLBGVO2RLQCKJ5KH4W54R20BG5BCECQT5C1LRAWVZNTVS' # your Foursquare Secret
VERSION = '20200311' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RMXFBEIKQ4VQJO50BKBC5XXEMCATBDENIXCDJG1GTFJWSSIH
CLIENT_SECRET:AVJVLBGVO2RLQCKJ5KH4W54R20BG5BCECQT5C1LRAWVZNTVS


### First capital's name

In [7]:
capital.loc[0, 'capital']

'Tokyo'

### First capital's Lat and Long

In [8]:
capital_latitude = capital.loc[0, 'lat'] # capital latitude value
capital_longitude = capital.loc[0, 'lng'] # capital longitude value

capital_name = capital.loc[0, 'capital'] # capital name

print('Latitude and longitude values of {} are {}, {}.'.format(capital_name, 
                                                               capital_latitude, 
                                                               capital_longitude))

Latitude and longitude values of Tokyo are 35.685, 139.7514.


### 200 venues that are within a radius of 5000 meters of first capital

In [14]:
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=food'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    capital_latitude, 
    capital_longitude, 
    radius, 
    LIMIT)

### Send the GET request and examine the results

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5e69c457c8cff2001b4f7055'},
 'response': {}}

### Foursquare get_category_type

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean json file and structure it into pandas dataframe

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

### Function to repeat the same process to all the capital


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Capital', 
                  'Capital Latitude', 
                  'Capital Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each capital and create a new dataframe called capital_venues


In [ ]:
capital_venues = getNearbyVenues(names=capital['capital'],
                                   latitudes=capital['lat'],
                                   longitudes=capital['lng'])

### Check the size of the resulting dataframe

In [ ]:
print(capital_venues.shape)
capital_venues.head()

In [ ]:
temp=capital_venues.groupby('Capital').count()
temp=temp.reset_index()
capital_venue_total=temp[['Capital','Venue']]
capital_venue_total.sort_values('Venue', ascending=False)

In [ ]:
# create map of world using latitude and longitude values
map_world = folium.Map(location=[0, 0], zoom_start=3)


# add markers to the map
markers_colors = []
for lat, lng, name in zip(capital_venues['Venue Latitude'], capital_venues['Venue Longitude'], capital_venues['Venue']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        clustered_marker = True,
        parse_html=False).add_to(map_world)
  
    
map_world

In [ ]:
SF_COORDINATES = (0,0)

 
# for speed purposes
MAX_RECORDS = 12138
  
# create empty map zoomed in on San Francisco
map2 = folium.Map(location=SF_COORDINATES, zoom_start=3)
 
# add a marker for every record in the filtered data, use a clustered view
for i in range(0,12139):
    folium.Marker(capital_venues.iloc[i]['Venue Latitude'], capital_venues.iloc[i]['Venue Longitude']).add_to(map2)

  
display(map)


